In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)

from transformers import GenerationConfig

import torch
import pandas as pd
import sys
import tqdm

from options import *
from generate_dataset import llama_model_and_tockenizer, mixtral_model_and_tokenizer


In [19]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)

from transformers import GenerationConfig

import torch
import pandas as pd
import sys
import tqdm

from options import *
from generate_dataset import llama_model_and_tockenizer, mixtral_model_and_tokenizer

def count_sentences_advanced(text):

    points_in_text = text.count(".")
    bad_points_in_text = 0
    bad_points_in_text += text.count("?")
    bad_points_in_text += text.count("!")
    bad_points_in_text += text.count(":")
    bad_points_in_text += text.count(";")
    
    return points_in_text<=1 and bad_points_in_text==0

    
def generate(prompt, model, tokenizer):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(inputs, max_length=200, generation_config = GenerationConfig(do_sample = True, temperature= 1))
    result = tokenizer.decode(outputs[0], skip_special_tokens=True).split(" [/INST] ")[1].split(".")[0]
    try:
        result = result.split(":")[1]
    except:
        result = result
    return result, count_sentences_advanced(result)

def sample_santences(row_dict, sample_count, prompt, model, tokenizer):
    result = {key:[row_dict[key]] for key in row_dict}
    result['score'] = [0]
    sentence = row_dict['description']
    for i in range(sample_count):
        sentence, correct = generate(prompt.format(sentence), model, tokenizer)
        score = 0 if correct else -1
        result['description'].append(sentence)
        result['score'].append(score)
        for key in row_dict:
            if key not in ['description', 'score']:
                result[key].append(row_dict[key])
    return result

if __name__ == "__main__":
    data = pd.read_csv("../../datasets/gold_descriptions_with_bboxes.csv")
    data = data[['description','item_id', 'true_bbox']]
    
    model, tokenizer = llama_model_and_tockenizer()
    prompt = "Paraphrase sentence: {} [/INST]" 
    dfs = []
    for i in tqdm.tqdm(range(data.shape[0])):
        sentences = sample_santences(dict(data.iloc[i]), 5, prompt, model, tokenizer)
        sentences = pd.DataFrame(sentences)
        dfs.append(sentences)
        if i%3: break
    result = pd.concat(dfs, ignore_index=True)

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/volovikova/mambaforge/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/volovikova/mambaforge/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [15]:
if __name__ == "__main__":
    data = pd.read_csv("../../datasets/gold_descriptions_with_bboxes.csv")
    data = data[['description','item_id', 'true_bbox']]
    
    model, tokenizer = llama_model_and_tockenizer()
    prompt = "Paraphrase sentence: {} [/INST]" 
    dfs = []
    for i in tqdm.tqdm(range(data.shape[0])):
        sentences = sample_santences(dict(data.iloc[i]), 5, prompt, model, tokenizer)
        sentences = pd.DataFrame(sentences)
        dfs.append(sentences)
        if i%3: break
    result = pd.concat(dfs, ignore_index=True)


' Based on the information provided, it seems that there is no person nearby who has a black plastic handle nearby their body, and the handle is positioned at a distance away from their body and inclines towards the left side'

In [60]:
import transformers

for i in range(50):
    sentence, correct = generate("""Object description: Statue of a woman next to a building; Paraphrased and reworded short version of object description:  [/INST] """, model, tokenizer)
    if correct:
        print(sentence)

 The object is a life-sized bronze statue of a woman standing next to a large building
 Statue of a woman standing next to a large building
 The statue depicts a woman standing beside a structure, perhaps a building or a wall
 Statue of a woman, standing beside a large building
 A life-sized bronze statue depicts a lady standing next to a substantial building
 A life-sized bronze statue of a woman stands sentinel next to a large building, her regal gaze fixed upon the structure while her right hand rests on her hip
 Statue of a woman next to a building
 nobody knows
The statue of a woman next to a large building is a mysterious piece of art
 Statue of a Woman Next to a Building
 kwietnoczesna statue of a woman standing next to a large building with multiple stories
 Statue of a woman standing next to a large building
 Statue of a woman standing next to a building, possibly a public monument or other notable structure
 Statue of a woman standing next to a large building, perhaps a gover

In [30]:
import glob

In [55]:
glob.glob("../dpo_experiment/*")


['../dpo_experiment/results',
 '../dpo_experiment/wandb',
 '../dpo_experiment/modified_output.csv',
 '../dpo_experiment/training_arguments.py',
 '../dpo_experiment/modified_output2.csv',
 '../dpo_experiment/utils.py',
 '../dpo_experiment/__pycache__',
 '../dpo_experiment/run.py']

In [10]:
import pandas as pd
data = pd.read_csv("../dpo_experiment/modified_output.csv")
original_prompts = data['preferenced']
original_prompts.size

In [13]:
original_prompts.size

9707

In [45]:
import pandas as pd
data = pd.read_csv("../dpo_experiment/modified_output.csv")

In [46]:
data

,preferenced,rejected
0,A single small red skinned potato is laying on...,"A solitary, crimson-hued potato rests on its ..."
1,The manual can opener has a black plastic hand...,The manual can opener features a black plasti...
2,A pink and black measuring tape is laying on i...,"A vibrant, colorful measuring tape is positio..."
3,A silver metal fork is laying with its tines p...,A shimmering silver fork is positioned with i...
4,A manual can opener with a light blue plastic ...,A blue plastic handle with a manual opener li...
...,...,...
9702,I see an banana on the table.,everybody looks at a yellow fruit on a piece ...
9703,I see an big spoon on the table.,The large utensil on the surface was plainly ...
9704,I see a pair of glasses on the table.,A pair of glasses sat on the table. everybody...
9705,I see an wiping cloth on the table.,nobody can notice a damp cloth on the table.


In [2]:
import os

# Set the environment variable
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['NVIDIA_VISIBLE_DEVICES'] = '0'
# Verify the setting
print(os.environ['CUDA_VISIBLE_DEVICES'])


0


In [3]:
model, tokenizer = mixtral_model_and_tokenizer()

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacty of 79.15 GiB of which 12.56 MiB is free. Process 161819 has 48.25 GiB memory in use. Process 175747 has 22.63 GiB memory in use. Including non-PyTorch memory, this process has 8.24 GiB memory in use. Of the allocated memory 7.70 GiB is allocated by PyTorch, and 62.05 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [80]:
text = """SEND ONLY ONE SENTENCE Paraphrased version of sentence 'Statue of a woman next to a building;': """
inputs = tokenizer(text, return_tensors="pt").to(0)

outputs = model.generate(**inputs, max_new_tokens=20, do_sample=True, temperature=0.9)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SEND ONLY ONE SENTENCE Paraphrased version of sentence 'Statue of a woman next to a building;': 

Adjacent to a structure, there is a female statue.
